In [1]:
from utils import transcription_utils as utils
import warnings
from IPython.display import Markdown, display

In [2]:

# Disable SSL warnings (optional but not recommended in production)
warnings.filterwarnings("ignore")

In [3]:
# 1. Settings
AUDIO_DIR = "./resources/input/"
OUTPUT_DIR = "./resources/output/"
MODEL_SIZE = "medium"  # You can use tiny, base, small, medium, large
LANGUAGE = "spanish"

summary = utils.transcribe_audio(AUDIO_DIR, OUTPUT_DIR, MODEL_SIZE, LANGUAGE)

display(Markdown(summary))



🎧 Using audio file: ./resources/input\audio_sample.mp3
🧠 Transcribing with Whisper (medium)... please wait, this can take several minutes...
✅ Transcription completed.
📝 Transcription saved at ./resources/output/transcription.txt
🧾 Generating text summary using LLaMa 3...
📄 Summary saved at ./resources/output/summary.txt


**Reunión Médica - Resumen y Puntos Clave**

**Fecha:** No especificada
**Persona:** Paciente con dolor en el pecho

**Resumen:**
El paciente presentó un dolor en el pecho que se inició 30 minutos antes de la reunión. El dolor era una sensación de presión y dureza, similar a una "presencia" de dolor en el pecho. No había tenido episodios similares previamente. Durante la entrevista, también reportó dificultad respiratoria y sensación de fatiga.

**Puntos Clave:**

*   Dolor en el pecho que se inició 30 minutos antes de la reunión
*   Dolor similar a una "presencia" de dolor en el pecho
*   Dificultad respiratoria desde hace aproximadamente un día
*   Sensación de fatiga
*   No tenido episodios similares previamente
*   Historia de enfermedades crónicas: hipertensión y hipercolesterolemia
*   Consumo de alcohol, drogas recreativas y tabaco (20 años)

**Recomendaciones Futuras:**
Deberá ser remitido a un especialista para una evaluación más detallada.